In [ ]:
from collections import defaultdict
import requests
import csv

In [ ]:

import re
import string
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('vader_lexicon')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

import nltk
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax
import time

import requests
from bs4 import BeautifulSoup
import csv
from textblob import TextBlob
import re
from collections import defaultdict
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import yfinance as yf
import os
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
def preprocess_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A)

    tokens = word_tokenize(text)

    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    lemmatizer = WordNetLemmatizer()
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in stemmed_tokens]

    return " ".join(lemmatized_tokens)

In [ ]:
api_key = 'GxvFJaFLlG9NOdncyCQUQkXxaRe7nvA5'

sections_to_include = [
    "Corrections", "New York", "International Home", "U.S.", "Business Day",
    "Real Estate", "World", "Times Insider", "T Magazine",
    "Technology", "Magazine", "Your Money", "Obituaries"
]

articles_by_date = defaultdict(list)
for year in range(2024, 2025):

    for month in range(1, 6):
        url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={api_key}"
        print(url)
        response = requests.get(url)
        print(response)

        if response.status_code == 200:
            data = response.json()
            print(f"Status code: {response.status_code}")

            for article in data['response']['docs']:
                section_name = article['section_name']
                if section_name in sections_to_include:
                    date = article['pub_date'][:10]
                    headline = article['headline']['main']
                    articles_by_date[date].append(headline)

        else:
            print(f"Failed API. Status code: {response.status_code}")

with open(f'nyt_articles_2024.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Date', 'Headlines'])

    for date, headlines in articles_by_date.items():
        joined_headlines = " ".join(headlines)
        writer.writerow([date, preprocess_text(joined_headlines)])

# In the main code I have not used this multiple times and only used this once to get the simple file.
# The file is nyt_articles_io.csv which came using this code.

https://api.nytimes.com/svc/archive/v1/2024/1.json?api-key=KPdZEAPGHoazqGaFckkGm1dNlCxULV85
<Response [200]>
Status code: 200
https://api.nytimes.com/svc/archive/v1/2024/2.json?api-key=KPdZEAPGHoazqGaFckkGm1dNlCxULV85
<Response [200]>
Status code: 200
https://api.nytimes.com/svc/archive/v1/2024/3.json?api-key=KPdZEAPGHoazqGaFckkGm1dNlCxULV85
<Response [200]>
Status code: 200
https://api.nytimes.com/svc/archive/v1/2024/4.json?api-key=KPdZEAPGHoazqGaFckkGm1dNlCxULV85
<Response [200]>
Status code: 200
https://api.nytimes.com/svc/archive/v1/2024/5.json?api-key=KPdZEAPGHoazqGaFckkGm1dNlCxULV85
<Response [200]>
Status code: 200
